In [ ]:
import os
import datetime
import pandas as pd
import util.data_puller as api
import plotly.figure_factory as ff

apikey = os.environ['API_KEY']
base_url = base_url = os.environ['API_BASE_URL'] + '/' + os.environ['API_CHANNEL'] + '/' + os.environ['API_F']

history_size = 60
start = str(datetime.datetime.now()).split('.')[0]
end = str(datetime.datetime(2020, 1, 1))
print(start, end)

In [ ]:
df = api.apicallv3(history_size, base_url, apikey, str(start), str(end), False, True)

In [ ]:
corrs = df.corr()

figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True
)

figure.show()

In [ ]:
df

In [ ]:
df.describe()